In [1]:
!pip install pandas
!pip install selenium
!pip install Pyarrow

   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB 660.6 kB/s eta 0:00:18
   ---------------------------------------- 0.1/11.5 MB 1.1 MB/s eta 0:00:11
    --------------------------------------- 0.2/11.5 MB 1.2 MB/s eta 0:00:10
    --------------------------------------- 0.2/11.5 MB 1.0 MB/s eta 0:00:11
    --------------------------------------- 0.3/11.5 MB 1.2 MB/s eta 0:00:10
   - -------------------------------------- 0.3/11.5 MB 1.1 MB/s eta 0:00:11
   - -------------------------------------- 0.4/11.5 MB 1.2 MB/s eta 0:00:10
   - -------------------------------------- 0.4/11.5 MB 1.1 MB/s eta 0:00:10
   - -------------------------------------- 0.5/11.5 MB 1.2 MB/s eta 0:00:10
   -- ------------------------------------- 0.6/11.5 MB 1.3 MB/s eta 0:00:09
   -- ------------------------------------- 0.6/11.5 MB 1.2 MB/s eta 0:00:09
   -- ------------------------------------- 0.7/11.5 MB 1.3 MB/s eta 0:00:09
   -

In [20]:
# Import necessary libraries
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import time
import re

def extract_emails(website_url):
    browser.get(website_url)
    # Get the regular expressions pattern to match on
    email_pattern = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,4}"
    # Get the html content for the page
    html = browser.page_source
    # Get a list of emails
    emails = re.findall(email_pattern, html)
    # Get a unique array of the emails on the page
    unique_emails_arr = pd.DataFrame(columns=['email'],data=emails)['email'].unique()
    # Convert to string with a comma delimiting them
    unique_emails_str = ','.join(unique_emails_arr)
    return unique_emails_str


# Define a function to extract search results for a given search URL, location, and industry
def extract_search_results(search_url,location,industry):

    # Navigate to the search URL
    browser.get(search_url)
    
    # Wait for the page to load
    time.sleep(5)
    
    # Find the element to start scrolling from
    start_scroll_element = browser.find_element(By.CLASS_NAME,"hfpxzc")
    scroll_origin = ScrollOrigin.from_element(start_scroll_element)
    
    # Perform scrolling action to reveal more search results
    for i in range(5):
        print(i)
        ActionChains(browser).scroll_from_origin(scroll_origin,0,1500*(i+1)).perform()
        time.sleep(3)
        
    # Inner function to extract information about a specific place from its page
    def extract_place_info(place_url):
        browser.get(place_url)
        time.sleep(3)
        header_text = browser.find_element(By.CLASS_NAME,'lMbq3e').text
        header_list = header_text.split("\n")
        place_name = header_list[0]
        reviews_stars = header_list[1]
        num_of_reviews = header_list[2].strip('(').strip(')')
        place_description = header_list[3]
        
        # Extract additional information like address, phone, and website
        info_elements = browser.find_elements(By.CLASS_NAME, "CsEnBe")
        
        if english:
            for info_element in info_elements:
                aria_label = info_element.get_attribute("aria-label")
                if 'Address:' in aria_label:
                    address = info_element.text
                if 'Phone:' in aria_label:
                    phone = info_element.text
                if 'Website:' in aria_label:
                    website = info_element.get_attribute("href")
        else:  # If Spanish
            for info_element in info_elements:
                aria_label = info_element.get_attribute("aria-label")
                if 'Dirección:' in aria_label:
                    address = info_element.text.split('\n')[1]
                if 'Teléfono:' in aria_label:
                    phone = info_element.text
                if 'Sitio web:' in aria_label:
                    website = info_element.get_attribute("href")
        
        emails = extract_emails(website)
        
        # Compile the extracted information into a list
        place_info_list = [
            industry,
            location,
            place_name,
            place_description,
            reviews_stars,
            num_of_reviews,
            address,
            phone,
            emails,
            website
        ]
        print(place_info_list)
        return place_info_list    
    
    # Find all elements that represent places in the search results
    place_elements = browser.find_elements(By.CLASS_NAME, "hfpxzc")
    print(len(place_elements))
    
    # Extract the URLs of all places
    place_urls = [element.get_attribute("href") for element in place_elements]
    
    place_info_list = []
    
    # Loop through the URLs to extract information for each place
    for url in place_urls:
        try:
            temp_place_info_list = extract_place_info(url)
            place_info_list.append(temp_place_info_list)
        except Exception as e:
            print(f"Error extracting place with url: {url}. Error: {e}")
        
        # Stop if 20 places have been found
        if len(place_info_list) >= 20:
            print('Breaking out of the for loop because 20 places were found.')
            break
    browser.close()
    return place_info_list


# Prepare to compile information for multiple locations and industries
final_places_list = []

# Spanish
english = False

# Define the locations and industries to search for
locations = ['Boston, MA',] # 'San Francisco, CA','New York, NY','Jacksonville, FL','Portland, OR','Denver, CO']
industries = ['Dental services',]  # Add more industries to search for
# 'Physical Therapy',
# 'HVAC',
# 'Veterinary Services',
# 'Plumbing',
# 'Insurance Brokerages',
# 'Landscaping',
# 'Automotive Repair',
# 'Chiropractic']

options = Options()
options.add_argument("--headless")
options.add_experimental_option('prefs', {'intl.accept_languages': 'en,en_US'})  # Set language preferences

# Loop through each industry and location, perform searches, and compile results
for industry in industries:
    print(industry)
    for location in locations:  
        # Initialize the WebDriver for Chrome
        browser = webdriver.Chrome(options=options)
        
        try:
            print(location)
            search_query = f"{industry} in {location}"
            print(search_query)
            search_url = f"https://www.google.com/maps/search/{search_query}"
            combo_place_info_list = extract_search_results(search_url,location,industry)
            pd.DataFrame(combo_place_info_list).to_csv(f'{industry} Export.csv',index = False)
            final_places_list.extend(combo_place_info_list)
        except Exception as e:
            print(e)
        
        try:  # Solve the weird error that occurs when trying to close the browser
            browser.close()
        except Exception as e:
            print(e)

try:
    browser.quit()
except Exception as e:
    print(e)

# Create a DataFrame from the compiled list and save it to a CSV file
places_df = pd.DataFrame(columns=['Industry', 'Location', 'Name', 'Description', 'Stars (out of 5)', 'Number of Reviews', 'Address', 'Phone', 'Emails','Website'], data=final_places_list)
places_df.to_csv('Places Information.csv', index=False)

Dental services
Boston, MA
Dental services in Boston, MA
0
1
2
3
4
26
['Dental services', 'Boston, MA', 'AP Dental Center', 'Dentista', '4.8', '320', '\ue0c8\n231 Border St B, Boston, MA 02128, Estados Unidos', '\ue0b0\n+1 617-315-1515', '', 'http://www.apdentalcenter.com/']
['Dental services', 'Boston, MA', 'South Boston Dental Group', 'Dentista', '4.9', '748', '\ue0c8\n9 Channel Center St, Boston, MA 02210, Estados Unidos', '', '', 'https://www.southbostondental.com/']
Error extracting place with url: https://www.google.com/maps/place/Seaport+Dental+Associates+Boston/data=!4m7!3m6!1s0x89e37a83f1dbba67:0x59bad559aab9f6ee!8m2!3d42.3451947!4d-71.0418677!16s%2Fg%2F1td1k_n1!19sChIJZ7rb8YN644kR7va5qlnVulk?authuser=0&hl=es-419&rclk=1. Error: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome-headless-shell=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF727FB1502+60802]
	(No symbol) [0x00007FF727F2AC02]
	(No symbol) [0x00007FF727DE7CE4]
	(No symbol) [0x00007

In [37]:
len(final_places_list)

fixed_addresses = []

for address in places_df['Address']:
    fixed_addresses.append(address.split('\n')[1])

In [39]:
fixed_addresses

['231 Border St B, Boston, MA 02128, Estados Unidos',
 '9 Channel Center St, Boston, MA 02210, Estados Unidos',
 '400 Commonwealth Ave 3rd floor, Boston, MA 02215, Estados Unidos',
 '1 Huntington Ave B, Boston, MA 02116, Estados Unidos',
 '1 McKinley Square Lobby B, Boston, MA 02109, Estados Unidos',
 '575 Boylston St 7th floor, Boston, MA 02116, Estados Unidos',
 '177 Tremont St #6, Boston, MA 02111, Estados Unidos',
 '77 Beacon St, Boston, MA 02108, Estados Unidos',
 '31 State St 7th Floor, Boston, MA 02109, Estados Unidos',
 '1610 Tremont St, Boston, MA 02120, Estados Unidos',
 '37 Newbury St #3, Boston, MA 02116, Estados Unidos',
 '36 Chauncy St, Boston, MA 02111, Estados Unidos',
 '591 E Broadway, Boston, MA 02127, Estados Unidos',
 '1313 Washington St, Boston, MA 02118, Estados Unidos',
 '272 W Broadway, Boston, MA 02127, Estados Unidos',
 '800 Boylston St 2nd floor, Boston, MA 02199, Estados Unidos',
 '22 Traveler St, Boston, MA 02118, Estados Unidos',
 '46 Farnsworth St, Boston

In [44]:
places_df.to_csv('Places Information Fixed.csv', index=False)